<a href="https://colab.research.google.com/github/chloesung/Econometrics/blob/main/1.%20Panel%20Data%20Methods/Wooldridge/Wooldridge_Ch14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
pip install wooldridge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 5.2 MB/s 


In [2]:
pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.3 MB/s 
     |████████████████████████████████| 9.8 MB 34.0 MB/s 
     |████████████████████████████████| 68 kB 4.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
import pandas as pd
import numpy as np
import re
import math

import seaborn as sns
sns.set_palette("pastel")
sns.set(style="whitegrid")
import warnings
import matplotlib.pyplot as plt
pal = sns.color_palette('Set1')

from scipy.stats import t
from statsmodels.formula.api import ols
pd.options.display.float_format = '{:.5f}'.format
plt.rcParams['axes.unicode_minus'] = False
import wooldridge
from linearmodels import PanelOLS
from linearmodels import RandomEffects

warnings.filterwarnings(action='ignore')
plt.rc('font', family='NanumBarunGothic') 
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina' #화질 좋게 해주기")

# Wage

## a) EDA

In [4]:
# Data Import
df = wooldridge.data('wagepan')[['nr','year','lwage','educ','black','hisp',
                                 'exper', 'expersq', 'union','married',
                                 'd81','d82','d83','d84','d85','d86','d87']]
df

,nr,year,lwage,educ,black,hisp,exper,expersq,union,married,d81,d82,d83,d84,d85,d86,d87
0,13,1980,1.19754,14,0,0,1,1,0,0,0,0,0,0,0,0,0
1,13,1981,1.85306,14,0,0,2,4,1,0,1,0,0,0,0,0,0
2,13,1982,1.34446,14,0,0,3,9,0,0,0,1,0,0,0,0,0
3,13,1983,1.43321,14,0,0,4,16,0,0,0,0,1,0,0,0,0
4,13,1984,1.56813,14,0,0,5,25,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4355,12548,1983,1.59188,9,0,0,8,64,0,1,0,0,1,0,0,0,0
4356,12548,1984,1.21254,9,0,0,9,81,1,1,0,0,0,1,0,0,0
4357,12548,1985,1.76596,9,0,0,10,100,0,1,0,0,0,0,1,0,0
4358,12548,1986,1.74589,9,0,0,11,121,1,1,0,0,0,0,0,1,0


- **time-constant:** $educ, black, hisp$
- **time-varying:** $exper, exper^2,  union, married$

## b) Pooled OLS
- $\beta_1$ should be negative.


In [5]:
# Regression
res = ols('lwage ~  educ + black + hisp + exper + expersq + married + union + d81 + d82 + d83 + d84 + d85 + d86 + d87', data = df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.187
Method:                 Least Squares   F-statistic:                     72.46
Date:                Tue, 26 Jul 2022   Prob (F-statistic):          7.25e-186
Time:                        07:28:40   Log-Likelihood:                -2982.0
No. Observations:                4360   AIC:                             5994.
Df Residuals:                    4345   BIC:                             6090.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0921      0.078      1.176      0.2

## c) Random Effects
[linearmodels: random effect](https://bashtage.github.io/linearmodels/panel/examples/examples.html)

In [6]:
# Multi-indexing
df.index = [df.iloc[:,0], df.iloc[:,1]]
df

nr  year   lwage  educ  black  hisp  exper  expersq  union  \
nr    year                                                                  
13    1980     13  1980 1.19754    14      0     0      1        1      0   
      1981     13  1981 1.85306    14      0     0      2        4      1   
      1982     13  1982 1.34446    14      0     0      3        9      0   
      1983     13  1983 1.43321    14      0     0      4       16      0   
      1984     13  1984 1.56813    14      0     0      5       25      0   
...           ...   ...     ...   ...    ...   ...    ...      ...    ...   
12548 1983  12548  1983 1.59188     9      0     0      8       64      0   
      1984  12548  1984 1.21254     9      0     0      9       81      1   
      1985  12548  1985 1.76596     9      0     0     10      100      0   
      1986  12548  1986 1.74589     9      0     0     11      121      1   
      1987  12548  1987 1.46654     9      0     0     12      144      1   

            married  d81  d82  d83  d84  d85  d86  d87  
nr    year                                              
13    1980        0    0    0    0    0    0    0    0  
      1981        0    1    0    0    0    0    0    0  
      1982        0    0    1    0    0    0    0    0  
      1983        0    0    0    1    0    0    0    0  
      1984        0    0    0    0    1    0    0    0  
...             ...  ...  ...  ...  ...  ...  ...  ...  
12548 1983        1    0    0    1    0    0    0    0  
      1984        1    0    0    0    1    0    0    0  
      1985        1    0    0    0    0    1    0    0  
      1986        1    0    0    0    0    0    1    0  
      1987        1    0    0    0    0    0    0    1  

[4360 rows x 17 columns]

In [7]:
# Exogenous Variables
exog_vars = ['educ', 'black', 'hisp', 'exper', 'expersq', 'married', 'union', 'd81', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87']
exog = df[exog_vars]
exog['const'] = 1

In [8]:
# Run regression
mod = RandomEffects(df.lwage, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:              RandomEffects   R-squared (Between):              0.1853
No. Observations:                4360   R-squared (Within):               0.1799
Date:                Tue, Jul 26 2022   R-squared (Overall):              0.1828
Time:                        07:28:40   Log-likelihood                   -1622.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      68.409
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             68.409
                            

## d) Fixed effects

In [9]:
# Exogenous Variables
exog_vars = ['expersq', 'married', 'union', 'd81', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87']
exog = df[exog_vars]
exog['const'] = 1

In [10]:
# Run regression
mod = PanelOLS(df.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Tue, Jul 26 2022   R-squared (Overall):              0.0807
Time:                        07:28:40   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            